<a href="https://colab.research.google.com/github/dude123studios/AdvancedDeepLearning/blob/main/Using_Conv1D's_for_better_text_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, preprocessing

max_len = 200
n_words = 10000
dim_embedding = 256
EPOCHS = 20
BATCH_SIZE = 500


In [3]:
def load_data():
    (x_train,y_train),(x_test,y_test) = datasets.imdb.load_data(num_words=n_words)
    x_train = preprocessing.sequence.pad_sequences(x_train,maxlen=max_len)
    x_test = preprocessing.sequence.pad_sequences(x_test,maxlen=max_len)
    return (x_train,y_train),(x_test,y_test)

In [4]:
def build_model():

    model = models.Sequential()
    model.add(layers.Embedding(n_words,dim_embedding,input_length=max_len))
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv1D(256,3,padding='valid',activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model


In [5]:
(x_train,y_train),(x_test,y_test) = load_data()


17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [7]:
with tf.device('gpu:0'):
    model = build_model()
    model.summary()
    score = model.fit(x_train,y_train,batch_size=BATCH_SIZE,
                      epochs=EPOCHS,validation_data=(x_test,y_test))
    score = model.evaluate(x_test,y_test,batch_size=BATCH_SIZE)
    print('\nTest score:',score[0])
    print('Test accuracy:',score[1])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 256)          2560000   
_________________________________________________________________
dropout (Dropout)            (None, 200, 256)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 198, 256)          196864    
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1